In [1]:
!pip3 install xrpl-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [2]:
from xrpl.asyncio.clients import AsyncJsonRpcClient
JSON_RPC_URL = "https://s.altnet.rippletest.net:51234/"
client = AsyncJsonRpcClient(JSON_RPC_URL)


In [3]:
from xrpl.models import ServerInfo
from pprint import pprint
pprint(await client.request(ServerInfo()))

Response(status=<ResponseStatus.SUCCESS: 'success'>,
         result={'info': {'build_version': '2.0.0-rc6',
                          'complete_ledgers': '27224048-44329144',
                          'hostid': 'FLAK',
                          'initial_sync_duration_us': '102209392',
                          'io_latency_ms': 1,
                          'jq_trans_overflow': '0',
                          'last_close': {'converge_time_s': 2, 'proposers': 6},
                          'load_factor': 1,
                          'network_id': 1,
                          'peer_disconnects': '97864',
                          'peer_disconnects_resources': '6810',
                          'peers': 100,
                          'ports': [{'port': '2459', 'protocol': ['peer']},
                                    {'port': '51233',
                                     'protocol': ['ws2', 'wss2']},
                                    {'port': '50051', 'protocol': ['grpc']}],
              

In [4]:
from xrpl.asyncio.wallet import generate_faucet_wallet
wallet = await generate_faucet_wallet(client)
print(wallet)

public_key: EDDA5A0D94D7835F412267A9CB1517FC4CBEE84601B90290377850646F90532E95
private_key: -HIDDEN-
classic_address: rEpPjFRjcFZN7MJhGup9NMF9haUZBrBTVK


In [5]:
from xrpl.models import Payment
from xrpl.utils import xrp_to_drops #1 mil drops in 1 xrp
from xrpl.wallet import Wallet
wallet2 = Wallet.create()
payment =  Payment(account=wallet.classic_address, amount=xrp_to_drops(23), destination=wallet2.classic_address)
pprint(payment) #sending a mock transaction of 23 xrp from one wallet to another

Payment(account='rEpPjFRjcFZN7MJhGup9NMF9haUZBrBTVK',
        transaction_type=<TransactionType.PAYMENT: 'Payment'>,
        fee=None,
        sequence=None,
        account_txn_id=None,
        flags=0,
        last_ledger_sequence=None,
        memos=None,
        signers=None,
        source_tag=None,
        signing_pub_key='',
        ticket_sequence=None,
        txn_signature=None,
        network_id=None,
        amount='23000000',
        destination='rpvsQmvcstegUhCX72ZfHpu91MpLBqbPEQ',
        destination_tag=None,
        invoice_id=None,
        paths=None,
        send_max=None,
        deliver_min=None)


In [33]:
#Our wallet has 23,000,000 drops now or 23 xrp :D

# Sending transactions between wallets
documentation: https://xrpl-py.readthedocs.io/en/stable/source/snippets.html#send-a-transaction-and-see-if-it-gets-validated

In [34]:
from xrpl.asyncio.account import get_balance
from xrpl.asyncio.clients import AsyncJsonRpcClient
from xrpl.models import Payment, SetRegularKey
from xrpl.asyncio.transaction import submit_and_wait
from xrpl.asyncio.wallet import generate_faucet_wallet
#loan amount can be set to payment amount between loaner and loanee's wallet.
#


In [35]:
wallet1 = await generate_faucet_wallet(client, debug =True)
wallet2 = await generate_faucet_wallet(client, debug=True)
regular_key_wallet = await generate_faucet_wallet(client, debug=True)

print("Balances before payment:")
print(await get_balance(wallet1.classic_address, client))
print(await get_balance(wallet2.classic_address, client))

Attempting to fund address rhiKsgjXCkWVwqA49HJiTBjKrZFdKoMH5A
Faucet fund successful.
Attempting to fund address rz8Ntx75iBAGQ9qLtQTvowGBvr5uKT4dN
Faucet fund successful.
Attempting to fund address rJG8tTSAWHHJzrvnttbeQe2B1JR9RUeJ1r
Faucet fund successful.
Balances before payment:
10000000000
10000000000


In [36]:
# Assign key pair (regular_key_wallet) to wallet1 using SetRegularKey transaction
tx = SetRegularKey(account=wallet1.address, regular_key=regular_key_wallet.address)

In [37]:
set_regular_key_response = await submit_and_wait(tx, client, wallet1)

print("Response for successful SetRegularKey tx:")
print(set_regular_key_response)

Response for successful SetRegularKey tx:
Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'Account': 'rhiKsgjXCkWVwqA49HJiTBjKrZFdKoMH5A', 'Fee': '10', 'Flags': 0, 'LastLedgerSequence': 44332419, 'RegularKey': 'rJG8tTSAWHHJzrvnttbeQe2B1JR9RUeJ1r', 'Sequence': 44332393, 'SigningPubKey': 'ED9F223B9ABE8CB3BB35C2D0DF3FACC0E5E20D038199890BF2698C0DF996392727', 'TransactionType': 'SetRegularKey', 'TxnSignature': '609D7E57C2E0D90FD900D3EF5457DA42722BA96CECED9B83E971A6699BC5799FA2823FD529E10A6BF3D61771030CF1CFC390871DD8DABA46830C694812716F01', 'ctid': 'C2A4757100120001', 'date': 757887470, 'hash': '1C625633B07CCDE2D0598CBF61594EA605DE95DF2A1E627FDEBAC9AD443F0AD1', 'inLedger': 44332401, 'ledger_index': 44332401, 'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Account': 'rhiKsgjXCkWVwqA49HJiTBjKrZFdKoMH5A', 'Balance': '9999999990', 'Flags': 65536, 'OwnerCount': 0, 'RegularKey': 'rJG8tTSAWHHJzrvnttbeQe2B1JR9RUeJ1r', 'Sequence': 44332394}, 'LedgerEntryType': 'AccountRoot'

In [38]:
# Since regular_key_wallet is linked to wallet1,
# walet1 can send payment to wallet2 and have regular_key_wallet sign it
payment =  Payment(
    account=wallet1.address,
    destination=wallet2.address,
    amount="1000",
)

In [39]:
payment_response = await submit_and_wait(payment, client, regular_key_wallet)


In [41]:
print("Response for tx signed using Regular Key:")
pprint(payment_response)

# Balance after sending 1000 from wallet1 to wallet2
print("Balances after payment:")
print(await get_balance(wallet1.address, client))
print(await get_balance(wallet2.address, client))

Response for tx signed using Regular Key:
Response(status=<ResponseStatus.SUCCESS: 'success'>,
         result={'Account': 'rhiKsgjXCkWVwqA49HJiTBjKrZFdKoMH5A',
                 'Amount': '1000',
                 'DeliverMax': '1000',
                 'Destination': 'rz8Ntx75iBAGQ9qLtQTvowGBvr5uKT4dN',
                 'Fee': '10',
                 'Flags': 0,
                 'LastLedgerSequence': 44332423,
                 'Sequence': 44332394,
                 'SigningPubKey': 'EDAB8631A56B44DAE15747DE89FD87394DBD4D08FC13F66E0C9A6E4321C39C852B',
                 'TransactionType': 'Payment',
                 'TxnSignature': 'BCD21E57B15D4C86E97EE498CD364E0DD2F7E9DE9590728485FE6C2C9CC3CB48A3CFD0C6B6F41334D9D0DCA5F8F8864A781B1B9D267A59BA51E072D27547E103',
                 'ctid': 'C2A4757500000001',
                 'date': 757887481,
                 'hash': '6B11DE0C011CA90ACBD84030C1ACAA05990DCE735216A1F13AD897EE56B66D3C',
                 'inLedger': 44332405,
                 'le